Simulation is from:
http://astrowiki.ph.surrey.ac.uk/dokuwiki/doku.php?id=tests:collision:mock_data

In [ ]:
from io import StringIO

import matplotlib
import matplotlib.pyplot as pl
pl.style.use('apw-notebook')
%matplotlib inline

import astropy.units as u
import numpy as np
from scipy.interpolate import interp1d
from pygaia.errors.photometric import gMagnitudeError, bpMagnitudeError, rpMagnitudeError
from pygaia.errors.astrometric import parallaxErrorSkyAvg, properMotionErrorSkyAvg
from pygaia.photometry.transformations import gminvFromVmini

from gary.observation import distance, distance_modulus

In [ ]:
D = 16*u.kpc
DM = distance_modulus(D)

Each line:
mass (Msun), three coordinates of position (pc), three components of velocity (km/s), the type of the star (Hurley, Pols & Tout 2000, MNRAS, 315, pp.546-7), the luminosity (Lsun), the stellar radius (Henon units), M_V, B-V.

In [ ]:
files = np.genfromtxt("/Users/adrian/Downloads/files.dat", dtype=None,
                      names=['time','total_N','Npairs','x','y','z'])
N_read = files['total_N'] - files['Npairs']
N_read = N_read[np.abs(files['time'] - 12023.).argmin()]

In [ ]:
names = ['mass', 'x','y','z', 'vx','vy','vz', 'stype', 'Lum', 'radius', 'M_V', 'B-V']
d = np.genfromtxt("/Users/adrian/Downloads/m4_simulation_12023.9.dat", names=names, dtype=None,
                  max_rows=N_read)

In [ ]:
d = d[d['stype'] < 10] # eliminate WDs, BHs, NSs, etc.

### Transform B-V to V-Ic using a hack from http://www.stsci.edu/~inr/intrins.html

In [ ]:
color_transform_tbl = np.genfromtxt("/Users/adrian/Downloads/cousins.txt", usecols=[2,4])
BV_to_VI = interp1d(color_transform_tbl[:,0], color_transform_tbl[:,1], kind='linear', bounds_error=False)
V_I = BV_to_VI(d['BV'])

M_G = d['M_V'] + gminvFromVmini(V_I)
m_G = M_G + DM

In [ ]:
pl.plot(d['BV'], V_I, ls='none')

---

Gaia errors

In [ ]:
dmG = gMagnitudeError(m_G)
pm_err = properMotionErrorSkyAvg(m_G, V_I) * u.microarcsecond/u.yr
pm_err = np.sqrt(pm_err[0]**2 + pm_err[1]**2).to(u.mas/u.yr)

In [ ]:
obs_m_G = np.random.normal(m_G, dmG)
# obs_V_I = 

# TODO: for color, estimate SDSS g-r, LSST g-r, WFIRST ??

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(8,5),sharex=True,sharey=True)

axes[0].plot(V_I, m_G, ls='none', marker='.', alpha=0.2)
axes[1].plot(obs_V_I, obs_m_G, ls='none', marker='.', alpha=0.2)

axes[0].set_xlim(-0.25,1.7)
axes[0].set_ylim(27.5,12)

In [ ]:
obs_m_G = np.random.normal(m_G, d_m_G)
obs_V_I = np.random.normal(V_I, np.sqrt(2)*d_m_G)

TODO: tangential velocity error will depend on distance error - maybe make plot for 5 tracers identified instead?

In [ ]:
vtan_err = (D*pm_err).to(u.km/u.s, equivalencies=u.dimensionless_angles()).value
idx = vtan_err < 25.

In [ ]:
pl.figure(figsize=(6,8))

pl.scatter(obs_V_I[idx], obs_m_G[idx], c=vtan_err[idx], cmap='viridis',
           vmin=1E-1, vmax=25, norm=matplotlib.colors.LogNorm(),
           marker='o', alpha=0.1)

pl.colorbar()
pl.xlim(-0.25,1.7)
pl.ylim(27.5,12)